# Battle of the Neighbourhoods week 3

This notebook will be used for the Applied Data Science Capstone project.
We will be exploring Four Squares geographic database to help a hypothetic job applicant find a neighbourhood on the other side of town which best matches their current neighbourhood in terms of ammenities.

### This week we will build a dataset of Toronto Neighbourhoods and there locations.

First, let's download and umport required modules

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


### Q1. Scrape Toronto postal codes from Wikipedia

In [3]:
# request contents of url
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# convert url table to dataframe        
TPC = pd.read_html(url)
hood_df = TPC[0]
hood_df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Next, we drop all rows where 'Borough' is not assigned

In [4]:
# create a list of indexes for all unassigned borouhs
Nas = hood_df[hood_df['Borough'] == 'Not assigned'].index

# drop all rows with unassigned boroughs
hood_df.drop(Nas, inplace=True)       
hood_df.reset_index(drop=True, inplace=True)

hood_df.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Let's get the shape of our dataframe.

In [5]:
hood_df.shape

(103, 3)

### Q2. Get geospatial data for neighbourhoods and merger them with the main dataframe.

Let's get the latitude and longitude for each neighbourhood and add them to our dataframe.

In [6]:
# Install and import geocoder module
#!pip install geocoder  # uncomment if geocoder not installed
import geocoder # fetches geospatial coordinates from address

I tried to loop through all postal codes in the list to return coordinates, but geocoder wasn't working. Uncomment this code if you want to test it.

In [ ]:
# # loop postal code instance until you get the coordinates
# def geo_loop(hood):
#     # initialize your variable to None
#     lat_lng_coords = None
#     while(lat_lng_coords is None):
#       g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#       lat_lng_coords = g.latlng

#     latitude = lat_lng_coords[0]
#     longitude = lat_lng_coords[1]

# # initialize geospatial dataframe
# geo_df = {'Postal Code': [], 'Latitude': [], 'Longitude': []}

# # loop through each postal code in hoods and return geospatial 
# # locations to geospatial dataframe
# for hood in hood_df:
#     geo_loop(hood)
#     geo_df['Latitude'].append(latitude)
#     geo_df['Longitude'].append(longitude)
    

Since geocoder isn't working we will download the provided CSV file.

In [8]:
# import geospatial data
geo_df = pd.read_csv('https://cocl.us/Geospatial_data')

geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then we will merge Latitude and Longitude into our Neighbourhood dataframe.

In [9]:
hood_df = pd.merge(hood_df, geo_df, on='Postal Code')

hood_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


### Q3. Explore and cluster the neighbourhoods of Toronto

Use geopy to get latitude and longitude of Toronto

In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinates of Toronto, Onatario are ' + str(latitude) + ',' + str(longitude) + '.')

The geographical coordinates of Toronto, Onatario are 43.6534817,-79.3839347.


Create a map of Toronto with neighbourhoods superimposed

In [11]:
# create a map of Toronto using coordinate values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add neighbourhood markers
for lat, lng, borough, neighbourhood in zip(hood_df['Latitude'], hood_df['Longitude'], hood_df['Borough'], hood_df['Neighbourhood']):
    #label = '{}, {}'.format(neighbourhood, borough)
    label = neighbourhood +',' + borough
    label = folium. Popup(label, parse_html=True)
    folium. CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)
                                                   
map_toronto

Define Foursquare Credentials and Version

In [12]:
CLIENT_ID = 'KVV0J33R0XIAZZMINTQ0ZCYLOEDBFILZA3XDAMM1J5Z1EBMS' # your Foursquare ID
CLIENT_SECRET = 'BOSAUFCFN5GHV5XRR4OWYGECAKBLUOHFDOSOETBD1MRSKG2T' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KVV0J33R0XIAZZMINTQ0ZCYLOEDBFILZA3XDAMM1J5Z1EBMS
CLIENT_SECRET:BOSAUFCFN5GHV5XRR4OWYGECAKBLUOHFDOSOETBD1MRSKG2T


Write a function to find the nearby venues for a given neighbourhood

In [28]:
# 
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Loop through the above function for each neighbourhood and save them to a dataframe

In [29]:
toronto_venues = getNearbyVenues(names = hood_df['Neighbourhood'],
                                latitudes = hood_df['Latitude'],
                                longitudes = hood_df['Longitude']
                                )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

Check the size of the resulting dataframe

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(2131, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Parkwoods,43.753259,-79.329656,TTC stop - 44 Valley Woods,43.755402,-79.333741,Bus Stop
3,Parkwoods,43.753259,-79.329656,Bella Vita Catering & Private Chef Service,43.756651,-79.331524,BBQ Joint
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


How many venues were returned for each neighbourhood?

In [25]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,58,58,58,58,58,58
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",15,15,15,15,15,15


How many unique categories can be found in the data?

In [30]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 269 unique categories.
